In [10]:
import os
import cv2
import numpy as np
import xml.etree.ElementTree as ET

In [5]:
def draw_bboxes_from_xml(xml_path, img_path, output_path):
    # Lade das Input-Bild, um Größe und Format zu übernehmen
    input_img = cv2.imread(img_path)
    if input_img is None:
        raise ValueError(f"Das Input-Bild konnte nicht geladen werden: {img_path}")
    
    # Erstelle ein neues Bild mit exakt derselben Form wie das Input-Bild, jedoch komplett schwarz
    out_img = np.zeros_like(input_img)
    
    # XML-Datei parsen
    tree = ET.parse(xml_path)
    root = tree.getroot()
    
    # Für jedes Objekt im XML: Bounding-Box-Punkte (x1 bis x4) extrahieren
    for obj in root.findall('object'):
        bndbox = obj.find('bndbox')
        x1 = int(float(bndbox.find('x1').text))
        y1 = int(float(bndbox.find('y1').text))
        x2 = int(float(bndbox.find('x2').text))
        y2 = int(float(bndbox.find('y2').text))
        x3 = int(float(bndbox.find('x3').text))
        y3 = int(float(bndbox.find('y3').text))
        x4 = int(float(bndbox.find('x4').text))
        y4 = int(float(bndbox.find('y4').text))
        
        # Erstelle ein Array der Eckpunkte
        pts = np.array([[x1, y1],
                        [x2, y2],
                        [x3, y3],
                        [x4, y4]], np.int32).reshape((-1, 1, 2))
        
        # Definiere die Farbe "weiß" entsprechend der Bildkanäle:
        # Bei einem Farb-/BGR-Bild: (255, 255, 255), bei einem Graustufenbild: 255.
        color = (255, 255, 255) if len(out_img.shape) == 3 and out_img.shape[2] == 3 else 255
        
        # Zeichne das Polygon als weiße Linie (Dicke 1 Pixel)
        cv2.polylines(out_img, [pts], isClosed=True, color=color, thickness=1)
    
    # Speichere das Ergebnis – das Output-Bild hat exakt dieselbe Größe und Format wie das Input-Bild
    cv2.imwrite(output_path, out_img)

In [14]:
def generate_bb_data(path = './wtw_data'):
    all_xml_files = os.listdir(path + '/xml')

    for _, file in enumerate(all_xml_files):
        xml_datei = file
        input_datei = file.replace('..', '.').replace('.xml', '.jpg')
        output_datei = input_datei

        try:
            draw_bboxes_from_xml(path + '/xml/' + xml_datei, path + '/images/' + input_datei, path + '/bbox_images/' + output_datei)
        except:
            continue

In [ ]:
# Uncomment to generate b/w images containing bounding boxes
# Takes > 2m
#generate_bb_data()

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
